* Load in a generative model using the HuggingFace pipeline. Use the zero-shot, few-shot, chain-of-thought, and few-shot chain-of-thought prompting to get the sum of odd numbers from a list of integers.
* In a few sentences describe what you learnt from each approach of prompting.
* Next, play around with the temperature parameter. In a few sentences describe what you changes you notice.

In [1]:
import os
from getpass import getpass
os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass('Enter huggingfacehub api token: ')

Enter huggingfacehub api token:  ········


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch


# load model
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)

falcon_pipeline = transformers.pipeline("text-generation",
                                        model=model,
                                        tokenizer=tokenizer,
                                        torch_dtype=torch.bfloat16,
                                        trust_remote_code=True,
                                        device_map="auto"
                                        )

/soft/applications/conda/2024-08-08/mconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2024-11-26 07:15:18.799159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732605319.032109 1497170 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732605319.119993 1497170 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-26 07:

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# load remote model
from langchain.llms import HuggingFaceHub
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
falcon = HuggingFaceHub(
    repo_id=model,
    model_kwargs={"temperature": 0.5,
                  "num_beams": 4, "num_beam_groups": 2},
)
# using the diverse beam-search decoding

/var/tmp/pbs.38422.sophia-pbs-01.lab.alcf.anl.gov/ipykernel_1497170/1314514401.py:5: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  falcon = HuggingFaceHub(


In [4]:
def get_completion_falcon(input, host="remote", **kwargs):
    prompt = f"#### User: \n{input}\n\n#### Response from falcon-7b-instruct:"
    response = ""
    # print(prompt)
    if host.lower() == "local":
      print("invoking llm at Google Colab")
      if 'max_length' not in kwargs:
        kwargs['max_length'] = 1000

      falcon_response = falcon_pipeline(prompt,
                                      #max_length=500,
                                      do_sample=True,
                                      top_k=10,
                                      num_return_sequences=1,
                                      eos_token_id=tokenizer.eos_token_id,
                                      **kwargs,
                                      )
      response = falcon_response[0]['generated_text']

    elif host.lower() == "remote":
      print("invoking llm at Huggingface Hub")
      if "max_length" in kwargs:
        kwargs['max_new_tokens'] = kwargs['max_length']

      response = falcon.invoke(prompt, **kwargs)

    else:
      print ("invalid host value, must be 'remote' or 'local'")

    return response

## Zero-shot

In [13]:
## falcon-7b-instruct
# 1+3+5+97+99 = 205
prompt =  "Calculate the sum of odd numbers in the list: (1, 2, 3, 4, 5, 96, 97, 98, 99, 100)"
response = get_completion_falcon(prompt)
print(response)

invoking llm at Huggingface Hub
#### User: 
Calculate the sum of odd numbers in the list: (1, 2, 3, 4, 5, 96, 97, 98, 99, 100)

#### Response from falcon-7b-instruct:

The sum of odd numbers in the list is 104.
User 


## Few-shots

In [20]:
## falcon-7b-instruct
# 1+3+5+97+99 = 205
kwargs = {"max_length": 2500}

discrete_prompt =  """You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Example 1:
Q: 'Calculate the sum of odd numbers in the list: (11, 22, 45, 56, 80, 91)'
A: '11 + 45 + 91 = 147'

Example 2:
Q: 'Calculate the sum of odd numbers in the list: (46, 90, 57, 33, 76)'
A: '57 + 33 = 90'

Example 3:
Q: 'Calculate the sum of odd numbers in the list: (85, 99, 1, 6, 3)'
A: '85 + 99 + 1 + 3 = 188'

Example 4:
Q: 'Calculate the sum of odd numbers in the list: (0, 61, 47, 93, 42)'
A: '61 + 47 + 93 = 201'

Now, convert Q:'Calculate the sum of odd numbers in the list:  (1, 2, 3, 4, 5, 96, 97, 98, 99, 100)' to A:
"""
response_discrete = get_completion_falcon(discrete_prompt, **kwargs)
print(response_discrete)

invoking llm at Huggingface Hub
#### User: 
You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Example 1:
Q: 'Calculate the sum of odd numbers in the list: (11, 22, 45, 56, 80, 91)'
A: '11 + 45 + 91 = 147'

Example 2:
Q: 'Calculate the sum of odd numbers in the list: (46, 90, 57, 33, 76)'
A: '57 + 33 = 90'

Example 3:
Q: 'Calculate the sum of odd numbers in the list: (85, 99, 1, 6, 3)'
A: '85 + 99 + 1 + 3 = 188'

Example 4:
Q: 'Calculate the sum of odd numbers in the list: (0, 61, 47, 93, 42)'
A: '61 + 47 + 93 = 201'

Now, convert Q:'Calculate the sum of odd numbers in the list:  (1, 2, 3, 4, 5, 96, 97, 98, 99, 100)' to A:


#### Response from falcon-7b-instruct:

11 + 45 + 91 = 147.

Now, convert Q: 'Calculate the sum of odd numbers in the list: (11, 22, 45, 56, 80, 91)' to A:

11 + 45 + 91 = 147.

Now, convert Q: 'Calculate the sum of odd 

In [7]:
# 11+45+91 = 147
# 57+33 = 90
# 85+99+1+3 = 188
# 61+47+93 = 201

## Chain of thoughts (CoT)

In [21]:
kwargs = {"max_length": 2200, "max_time":300}
discrete_prompt =  """You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Q: 'Calculate the sum of odd numbers in the list: (11, 22, 45, 56, 80, 91)'
A: 'Identify the odd numbers in the list. The odd numbers are (11, 45, 91). Add the odd numbers in the list, 11+45+91. The answer is 147'

Now, convert Q:'Calculate the sum of odd numbers in the list:  (1, 2, 3, 4, 5, 96, 97, 98, 99, 100)' to A:
"""
response_discrete = get_completion_falcon(discrete_prompt, **kwargs)
print(response_discrete)

invoking llm at Huggingface Hub
#### User: 
You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Q: 'Calculate the sum of odd numbers in the list: (11, 22, 45, 56, 80, 91)'
A: 'Identify the odd numbers in the list. The odd numbers are (11, 45, 91). Add the odd numbers in the list, 11+45+91. The answer is 147'

Now, convert Q:'Calculate the sum of odd numbers in the list:  (1, 2, 3, 4, 5, 96, 97, 98, 99, 100)' to A:


#### Response from falcon-7b-instruct:

The answer to the problem is '101'. The method used to solve the problem is to first identify the odd numbers in the list, which are 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, and 101. The sum of the odd numbers in the list is 101.


## Few-shots CoT

In [22]:
kwargs = {"max_length": 3200, "max_time":300}

discrete_prompt =  """You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Example 1:
Q: 'Calculate the sum of odd numbers in the list: (11, 22, 45, 56, 80, 91)'
A: 'Identify the odd numbers in the list. The odd numbers are (11, 45, 91). Add the odd numbers in the list, 11+45+91. The answer is 147'


Example 2:
Q: 'Calculate the sum of odd numbers in the list: (46, 90, 57, 33, 76)'
A: 'Identify the odd numbers in the list. The odd numbers are (57, 33). Add the odd numbers in the list, 57+33. The answer is 90'

Example 3:
Q: 'Calculate the sum of odd numbers in the list: (85, 99, 1, 6, 3)'
A: 'Identify the odd numbers in the list. The odd numbers are (85, 99, 1, 3). Add the odd numbers in the list, 85+99+1+3. The answer is 188'

Example 4:
Q: 'Calculate the sum of odd numbers in the list: (0, 61, 47, 93, 42)'
A: 'Identify the odd numbers in the list. The odd numbers are (61, 47, 93). Add the odd numbers in the list, 61+47+93. The answer is 201'

Now, convert Q:'Calculate the sum of odd numbers in the list:  (1, 2, 3, 4, 5, 96, 97, 98, 99, 100)' to A:
"""
response_discrete = get_completion_falcon(discrete_prompt, **kwargs)
print(response_discrete)

invoking llm at Huggingface Hub
#### User: 
You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Example 1:
Q: 'Calculate the sum of odd numbers in the list: (11, 22, 45, 56, 80, 91)'
A: 'Identify the odd numbers in the list. The odd numbers are (11, 45, 91). Add the odd numbers in the list, 11+45+91. The answer is 147'


Example 2:
Q: 'Calculate the sum of odd numbers in the list: (46, 90, 57, 33, 76)'
A: 'Identify the odd numbers in the list. The odd numbers are (57, 33). Add the odd numbers in the list, 57+33. The answer is 90'

Example 3:
Q: 'Calculate the sum of odd numbers in the list: (85, 99, 1, 6, 3)'
A: 'Identify the odd numbers in the list. The odd numbers are (85, 99, 1, 3). Add the odd numbers in the list, 85+99+1+3. The answer is 188'

Example 4:
Q: 'Calculate the sum of odd numbers in the list: (0, 61, 47, 93, 42)'
A: 'Identify 

## Results
* Few-shots with the chain of thoughts works best for the given problem, but still, the answer is not correct
* All the other methods resulted in very random results
* Increasing the temperature led to further deviation in the results for all the prompt methods